In [ ]:
import requests
from lxml import etree
import random
import time

user = ''

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    'Cookie': ''
}

movie_list = []
start = "0"
url1 = 'https://movie.douban.com/people/{}/collect?start={}&sort=time&rating=all&filter=all&mode=list'.format(user, start)
url2 = 'https://book.douban.com/people/{}/collect?sort=time&start={}&filter=all&mode=list'.format(user, start)
page_text = requests.get(url=url1, headers=headers).text
tree = etree.HTML(page_text)
movie_list_e = tree.xpath('/html/body/div[3]/div[1]/div[2]/div[1]/ul/li')
movie_list.append(movie_list_e)

while movie_list_e != []:
    start = str(int(start)+30)
    url1 = 'https://movie.douban.com/people/{}/collect?start={}&sort=time&rating=all&filter=all&mode=list'.format(user, start)
    url2 = 'https://book.douban.com/people/{}/collect?sort=time&start={}&filter=all&mode=list'.format(user, start)
    page_text = requests.get(url=url1, headers=headers).text
    tree = etree.HTML(page_text)
    movie_list_e = tree.xpath('/html/body/div[3]/div[1]/div[2]/div[1]/ul/li')
    movie_list.append(movie_list_e)
movie_list.pop()

In [ ]:
movie_url_list = []
movie_name_list = []
movie_star_list = []

for movie_list_e in movie_list:
    for movie in movie_list_e:
        movie_url = movie.xpath('./div[1]/div[1]/a/@href')[0]
        movie_url_list.append(movie_url)

        movie_name = movie.xpath('./div[1]/div[1]/a/text()')[0].strip()
        movie_name_list.append(movie_name)

        if movie.xpath('./div[1]/div[2]/span/@class') != []:
            movie_star = movie.xpath('./div[1]/div[2]/span/@class')[0]
            movie_star_list.append(movie_star)

    """ print(movie_url_list)
    print(movie_name_list)
    print(movie_star_list) """

    user_socre_list = []
    for i in movie_star_list:
        user_score = int(i[6]) * 2
        user_socre_list.append(user_score)

print("用户平均打分为：\n", sum(user_socre_list)/len(user_socre_list))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = user_socre_list
x = range(1, len(user_socre_list)+1)

plt.plot(x, data, label='折线图')
plt.show()
degree = 10
coefficients = np.polyfit(x, data, degree)
polynomial = np.poly1d(coefficients)
y_fit = polynomial(x)
plt.plot(x, data, 'o', label='原始数据')
plt.plot(x, y_fit, label='拟合曲线')
plt.show()

In [ ]:
douban_rating_list = []
for douban_url in movie_url_list:
    # time.sleep(random.randint(1, 4))
    url2 = douban_url
    page_text = requests.get(url=url2, headers=headers).text
    tree2 = etree.HTML(page_text)
    douban_rating = tree2.xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/strong/text()')
    if douban_rating != [] and douban_rating != ['  ']:
        douban_rating_list.append(eval(douban_rating[0]))

print("电影豆瓣平均分为：\n",sum(douban_rating_list)/len(douban_rating_list), end='')